### Rag Using Langchain

In [1]:
import json
import pandas as pd
import re
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.llms import Ollama
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import numpy as np
import faiss
from langchain.docstore import InMemoryDocstore
import json
from langchain_community.vectorstores import FAISS

In [2]:
top_k = 5
LLM_Model = Ollama(model='qwen3:8b', temperature=0.3, num_ctx=4096)

/var/folders/c2/f9lh6rmd4q1648_pfl1636zw0000gn/T/ipykernel_78148/824802379.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  LLM_Model = Ollama(model='qwen3:8b', temperature=0.3, num_ctx=4096)


In [3]:
def clean_text(text):
    """Clean text data"""
    text = str(text)
    text = re.sub(r'\n{3,}', '\n\n', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()



In [4]:
def load_and_preprocess_data(file_path):
    """Load and preprocess a single JSON file."""
    with open(file_path, 'r') as f:
        raw_data = json.load(f)
    clean_texts = [clean_text(entry) for entry in raw_data if isinstance(entry, str)]
    combined_text = "\n".join(clean_texts)
    return combined_text  # return raw text (not Document yet)


#  Load and tag each PDF with <NEW PDF>
pdf_texts = []
for idx, pdf_file in enumerate(["Market Research Report_extracted_text.json", 'PMS Market Research_extracted_text.json']):
    text = load_and_preprocess_data(pdf_file)
    tagged = f"<NEW PDF>\n{text.strip()}"
    pdf_texts.append(tagged)

#  Combine into one string for <NEW PDF> token splitting
combined_text = "\n".join(pdf_texts)

#  Split back into separate PDFs using the tag
pdf_docs = combined_text.split("<NEW PDF>")

#  Create Document objects per PDF
individual_documents = [
    Document(page_content=pdf.strip(), metadata={"pdf_id": i})
    for i, pdf in enumerate(pdf_docs) if pdf.strip()
]

#  Now split each document into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50, 
    length_function=lambda x: len(x.split()),
    separators=["\n\n\n", "\n\n", "\n", ". ", "! ", "? ", "; ", ", ", " ", ""],
    keep_separator=False,
    add_start_index=True,
    strip_whitespace=True
)

# 🧠 Chunk each PDF individually
chunked_docs = []
for doc in individual_documents:
    chunks = text_splitter.split_text(doc.page_content)
    for i, chunk in enumerate(chunks):
        chunked_docs.append(
            Document(page_content=chunk, metadata={"pdf_id": doc.metadata["pdf_id"], "chunk_id": i})
        )

print(f"✅ Total Chunks: {len(chunked_docs)}")


✅ Total Chunks: 26


In [5]:
chunks[-2]

'Renewable and Solar -Focused Platforms During our research, we also explored platforms that are specifically designed for the renewable energy sector and others that are solar PV -focused . At first, they seemed promising because they’re built with energy systems in mind. However, after testing and reviewing them, we found that they don’t meet the type of project management needs we’re aiming for , although they market them selves as they have project management tools. Platforms like Ra Power Management (RaPM) , SenseHawk, and Payac a are examples of solar - focused systems. These tools are mainly designed for monitoring the performance of solar plants — such as tracking electricity production, system health, faults, and maintenance alerts. While they’re excellent for operations and post -installation monitoring , they do not support document approvals, workflows, submittals, or collaboration between stakeholde rs like contractors, consultants, and clients. In short, these are more li

In [7]:
individual_documents[-1].page_content[:500]  # Show first 500 chars of last document

'MARKET RESEARCH REPORT ON PROJECT MANAGEMENT SYSTEMS Benchmarking Tools for Document Control, Approvals, and Team Collaboration in PV proje cts. Mahi nour Mohammad Abstract This report explores existing project management systems to identify gaps and opportunities for developing a platform tailored to photovoltaic (PV) projects.\nIn too many solar and construction projects, teams juggle emails, spreadsheets, and generic cloud folders just to get a simple drawing reviewed. The result? Lost version'

In [10]:
import torch

# 🔍 Detect best device
device = (
    'cuda' if torch.cuda.is_available() else
    'mps' if torch.backends.mps.is_available() else
    'cpu'
)

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': device}
)


# ✅ Embed in batches for efficiency
def batch_embed(texts, batch_size=64):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        batch_embeddings = embedding_model.embed_documents(batch)
        embeddings.extend(batch_embeddings)
    return np.array(embeddings, dtype=np.float32)


# 🧠 Build vectorstore from chunked Document objects
def create_vectorstore(docs):
    """
    Create a FAISS vector store from a list of Document objects.
    Each document should have metadata like pdf_id, chunk_id, etc.
    """
    texts = [doc.page_content for doc in docs]
    embeddings = batch_embed(texts)

    # Ensure FAISS Index is properly initialized
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    
    # Normalize embeddings for cosine similarity
    faiss.normalize_L2(embeddings)
    index.add(embeddings)

    # Map back to Documents using an in-memory docstore
    docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(docs)})
    index_to_docstore_id = {i: str(i) for i in range(len(docs))}

    # Use embed_query for similarity search
    vectorstore = FAISS(
        embedding_function=embedding_model.embed_query,
        index=index,
        docstore=docstore,
        index_to_docstore_id=index_to_docstore_id
    )

    return vectorstore


In [11]:
vectorstore = create_vectorstore(chunked_docs)
query = "What are the main findings from the market research?"
docs = vectorstore.similarity_search(query, k=5)
for d in docs:
    print(d.metadata, d.page_content[:200], "\n---")



`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


{'pdf_id': 2, 'chunk_id': 19} To synthesize these findings, a feature matrix has been developed that quantifies how well each platform supports a core set of functionalities —from role assignment and approval workflows to bid mana 
---
{'pdf_id': 2, 'chunk_id': 16} The owner and consultant evaluate price, timing, experience, and any value -adds. The contract is awarded to the bidder who best meets those criteria . Once signed, the real work begins: turning that  
---
{'pdf_id': 1, 'chunk_id': 1} Executive Summary This section provides a high -level overview of the findings from the market research. Key Findings • Doctranslator : Offers free services with good layout preservation for English t 
---
{'pdf_id': 1, 'chunk_id': 0} MARKET RESEARCH REPORT: ANALYSIS OF DOCUMENT TRANSLATION TOOLS Evaluating Leading Solutions for Multilingual Document Translation Mah inour Mohammad
Introduction This market research report analyzes c 
---
{'pdf_id': 2, 'chunk_id': 0} MARKET RESEARCH REPORT ON PRO

In [12]:
def save_vectorstore(vector_store, directory_path):
    """Save vector store to a directory."""
    vector_store.save_local(directory_path)

def load_vectorstore(directory_path, embeddings):
    """Load vector store from a directory."""
    return FAISS.load_local(directory_path, embeddings)


In [13]:
def format_docs(docs):
    """Format documents for context"""
    return "\n\n".join(
        f"[Source {i} | PDF {doc.metadata.get('pdf_id', '?')}]: {doc.page_content}"
        for i, doc in enumerate(docs, 1)
    )

def qa_chain(vector_store, top_k=5, llm=LLM_Model):
    """Setup a QA chain with custom prompts and vector retriever"""
    retriever = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": top_k}
    )

    print("✅ RETRIEVER SETUP COMPLETE")

    prompt_template = """You are a helpful assistant. Analyze the context and provide a structured response.

Context:
{context}

Question: {question}

Please provide your response in exactly this format:

RESPONSE:
[Your direct, concise answer to the question]

REASONING:
[Brief explanation of how you arrived at this answer using the sources]

SOURCES:
[List the source numbers that support your answer, e.g., 1, 2, 3]

Important: Do not include any <think> tags or internal reasoning. Be direct and concise."""

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    chain = (
        {
            "context": retriever | format_docs,
            "question": RunnablePassthrough()
        }
        | PROMPT
        | llm
        | StrOutputParser()
    )

    return chain


In [15]:
def parse_structured_response(response_text):
    """Parse the structured response"""
    cleaned_response = re.sub(r'<think>.*?</think>', '', response_text, flags=re.DOTALL)
    cleaned_response = re.sub(r'<[^>]+>', '', cleaned_response)
    cleaned_response = re.sub(r'\n\s*\n', '\n\n', cleaned_response.strip())
        
    sections = {'response': '', 'reasoning': '', 'sources': ''}
    current_section = None
    current_content = []
        
    lines = cleaned_response.split('\n')
        
    for line in lines:
        line = line.strip()
            
        if line.upper().startswith('RESPONSE:'):
            if current_section:
                sections[current_section] = '\n'.join(current_content).strip()
            current_section = 'response'
            current_content = [line[9:].strip()]
                
        elif line.upper().startswith('REASONING:'):
            if current_section:
                sections[current_section] = '\n'.join(current_content).strip()
            current_section = 'reasoning'
            current_content = [line[10:].strip()]
                
        elif line.upper().startswith('SOURCES:'):
            if current_section:
                sections[current_section] = '\n'.join(current_content).strip()
            current_section = 'sources'
            current_content = [line[8:].strip()]
                
        elif current_section and line:
            current_content.append(line)
        
    if current_section:
        sections[current_section] = '\n'.join(current_content).strip()
        
    source_ids = []
    if sections['sources']:
        source_text = sections['sources']
        source_ids = [int(x) for x in re.findall(r'\d+', source_text)]
        
    return {
            'answer': sections['response'],
            'reasoning': sections['reasoning'],
            'sources': source_ids,
            'raw_response': cleaned_response
        }
    

In [17]:
def ask_question(question, vector_store, top_k=5, return_sources=True):
    """Ask a question and optionally return source documents"""
    chain = qa_chain(vector_store, top_k=top_k)
    response = chain.invoke(question)
    

    parsed_response = parse_structured_response(response)
    
    if not parsed_response:
        return {"error": "Failed to parse LLM response.", "raw": response}


    if return_sources:
        retriever = vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": top_k}
        )
        source_docs = retriever.get_relevant_documents(question)

        parsed_response['answer'] = parsed_response.get('answer', '').strip()
        parsed_response['reasoning'] = parsed_response.get('reasoning', '').strip()
        parsed_response['sources'] = [int(x) for x in parsed_response.get('sources', [])]
        parsed_response['source_documents'] = source_docs
        parsed_response['source_texts'] = [doc.page_content for doc in source_docs]

    return parsed_response


In [19]:

response = ask_question("What are the key findings from the market research report?", vectorstore)
print(response['answer'])
print("\nReasoning:")
print(response['reasoning'])
print("\nSources:")
print(response['sources'])

✅ RETRIEVER SETUP COMPLETE
The key findings include: (1) No single document translation tool fully meets all requirements for Arabic, French, and English translations, particularly for OCR, layout preservation, and mixed-language content. (2) Procore excels in document management, versioning, and submittal workflows but lacks in bids management. (3) Platforms like PMWeb and Aconex outperform others in financial oversight and document approval workflows.

Reasoning:
The market research highlights gaps in translation tools (Source 2), Procore’s strengths in project management (Source 1 and 3), and comparative feature analyses (Source 1, 4, 5).

Sources:
[1, 2, 3, 4, 5]


/var/folders/c2/f9lh6rmd4q1648_pfl1636zw0000gn/T/ipykernel_78148/583521648.py:18: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  source_docs = retriever.get_relevant_documents(question)


In [21]:
ask_question(" If you had to build a hybrid solution using two platforms from the research, which combination would you choose for a $50M solar project, \
                      and how would you handle the integration challenges, particularly around \
                      the 12 core features identified in the research?", vectorstore, top_k=5, return_sources=True)

✅ RETRIEVER SETUP COMPLETE


{'answer': 'PMWeb and Aconex would be the optimal combination for a $50M solar project.',
 'reasoning': 'PMWeb excels in cost control, contract management, and workflow automation (Sources 1, 3, 5), while Aconex provides robust document control, collaboration, and centralized project management (Sources 1, 4). Together, they cover all 12 core features (Sources 2), including document approvals, versioning, financial oversight, and submittal tracking. Integration challenges would be addressed via APIs/middleware to synchronize data, ensure role-based access, and maintain version consistency.',
 'sources': [1, 2, 3, 4, 5],
 'raw_response': 'RESPONSE:  \nPMWeb and Aconex would be the optimal combination for a $50M solar project.  \n\nREASONING:  \nPMWeb excels in cost control, contract management, and workflow automation (Sources 1, 3, 5), while Aconex provides robust document control, collaboration, and centralized project management (Sources 1, 4). Together, they cover all 12 core featur

### Summerization trial

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.mapreduce import MapReduceDocumentsChain
from langchain.chains import StuffDocumentsChain
from langchain.chains.llm import LLMChain
# MAP
map_prompt = ChatPromptTemplate.from_template(
    "Summarize the following document chunk:\n\n{context}"
)
map_llm_chain = LLMChain(
    llm=LLM_Model,
    prompt=map_prompt,
    output_parser=StrOutputParser()
)

# REDUCE
reduce_prompt = ChatPromptTemplate.from_template(
    "Given these summaries, create a final concise summary:\n\n{context}"
)
reduce_llm_chain = LLMChain(
    llm=LLM_Model,
    prompt=reduce_prompt,
    output_parser=StrOutputParser()
)
reduce_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_llm_chain,
    document_variable_name="context"
)

# COMPOSE
summarize_chain = MapReduceDocumentsChain(
    llm_chain=map_llm_chain,
    reduce_documents_chain=reduce_documents_chain,
    document_variable_name="context",
    return_intermediate_steps=True
)

def summarize_per_pdf(pdf_docs):
    """Summarize each PDF independently using Map-Reduce"""
    summaries = []

    for i, doc in enumerate(pdf_docs):
        chunks = text_splitter.split_documents([doc])

        result = summarize_chain.invoke(chunks)
        final_summary = result["output_text"]

        summaries.append({
            "pdf_id": doc.metadata.get("pdf_id", i),
            "summary": final_summary,
            "intermediate_summaries": result["intermediate_steps"]
        })

    return summaries
summaries = summarize_per_pdf(individual_documents)

for entry in summaries:
    print(f"📄 PDF {entry['pdf_id']} Summary:\n{entry['summary']}\n{'='*40}")

📄 PDF 1 Summary:
<think>
Okay, I need to create a final concise summary based on the provided text. Let me start by reading through the entire content to understand the main points.

The text is a detailed comparison of various document translation tools, focusing on their features, pricing, and performance. The key sections include evaluations of specific tools like Doctranslator, Doctranslate.io, Doclingo, DeepL, and others. Each tool has its strengths and weaknesses, such as layout preservation, OCR support, language capabilities, and pricing models. There's also a mention of premium plans with different character limits and engine options. The summary at the end highlights the varied performance across criteria and the range of pricing from affordable to premium.

Now, I need to condense this into a concise summary. I should start by stating the purpose of the evaluation, then list the main tools and their key features, mention the pricing models, and conclude with the overall asse

## Q&A Module

In [35]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

qa_generation_prompt = ChatPromptTemplate.from_template(
    """You are a helpful assistant tasked with generating question-answer pairs for study purposes.

Text:
{context}

Generate 2-3 meaningful questions and their answers based only on the above text. Format your output exactly like this:

Q1: ...
A1: ...

Q2: ...
A2: ...

(Do not invent facts or go beyond the text.)
"""
)

qa_chain = LLMChain(
    llm=LLM_Model,
    prompt=qa_generation_prompt,
    output_parser=StrOutputParser()
)


In [36]:
def generate_qa_pairs(docs, batch_size=1):
    """Generate QA pairs from a list of Document chunks"""
    qa_pairs = []

    for doc in docs:
        try:
            output = qa_chain.invoke({"context": doc.page_content})
            qa_pairs.append({
                "pdf_id": doc.metadata.get("pdf_id"),
                "chunk_id": doc.metadata.get("chunk_id"),
                "text": doc.page_content,
                "qa_output": output.strip()
            })
        except Exception as e:
            print(f"❌ Failed to generate QA for chunk {doc.metadata}: {e}")

    return qa_pairs


In [ ]:
QA_pairs = generate_qa_pairs(chunked_docs)

### Stuff method

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate

# Define prompt
prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following:\\n\\n{context}")]
)

# Instantiate chain
chain = create_stuff_documents_chain(LLM_Model, prompt)

# Invoke chain with a list of Document objects
result = chain.invoke({"context": [document]})
print(result)

<think>
Okay, so I need to analyze this detailed comparison of project management platforms for the construction industry. Let me start by breaking down what each platform offers based on the information provided. 

First, the user mentioned platforms like Monday.com, Wrike, PMWeb, Aconex, and Procore. They also touched on renewable energy-focused tools like RaPM, SenseHawk, and Payaca, but those are more monitoring tools rather than project management systems. The main focus is on construction-specific needs.

Looking at the feature matrix, each platform is rated on 12 features. Let me go through each one:

1. **Assigning roles**: Monday.com is at 10%, which is low. Wrike is 85%, PMWeb 80%, Aconex 80%, Procore 80%. So Wrike and the others are better here. Maybe Monday.com lacks role-based task assignment.

2. **Document approval workflow**: All except Monday.com are 100% or 80% or 80%+. Wait, the matrix shows Monday.com at 60%, Wrike 100%, PMWeb 100%, Aconex 100%, Procore 100%. Wait, 

### LangChain chains

In [99]:
from langchain.chains.summarize import load_summarize_chain


chunk_docs = [Document(page_content=chunk) for chunk in chunks]

chain = load_summarize_chain(LLM_Model, chain_type="map_reduce", verbose=True)
map_reduce_summary = chain.run(chunk_docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"MARKET RESEARCH REPORT: ANALYSIS OF DOCUMENT TRANSLATION TOOLS Evaluating Leading Solutions for Multilingual Document Translation Mah inour Mohammad
Introduction This market research report analyzes competitors offering document translation tools that support PDF, Word, Excel, and scanned images while preserving layout and formatting. The focus is on tools that handle Arabic, French, and English languages, catering to both B2B and B2C markets. The key features evaluated include layout preservation, Arabic support and quality, translation accuracy and speed, pricing model, and Optical Character Recognition (OCR) support. To assess these tools, a series of test cases were conducted for each language, including: 1. Text -based documents: Evaluating basic translation accuracy, layout preservation, handling of number lists, bullet points,

Traceback (most recent call last):
  File "/Users/maryamsaad/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/c2/f9lh6rmd4q1648_pfl1636zw0000gn/T/ipykernel_35989/2302064687.py", line 7, in <module>
    map_reduce_summary = chain.run(chunk_docs)
  File "/Users/maryamsaad/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py", line 189, in warning_emitting_wrapper
    return wrapped(*args, **kwargs)
  File "/Users/maryamsaad/Library/Python/3.9/lib/python/site-packages/langchain/chains/base.py", line 603, in run
    return self(args[0], callbacks=callbacks, tags=tags, metadata=metadata)[
  File "/Users/maryamsaad/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py", line 189, in warning_emitting_wrapper
    return wrapped(*args, **kwargs)
  File "/Users/maryamsaad/Library/Python/3.9/lib/python/site-packages/lang

In [ ]:
print(map_reduce_summary)

<think>
Okay, the user wants a concise summary of the provided text. Let me start by reading through the original content carefully.

The main points seem to be about comparing project management platforms for the construction and renewable energy sectors. The text mentions that some platforms are focused on solar PV monitoring but lack project management features like workflow approvals and collaboration. Then there's a comparison of several platforms: Monday.com, Wrike, PMWeb, Aconex, and Pro, each with different strengths and weaknesses. The feature matrix is also part of the summary, showing how each platform scores on various functionalities.

I need to make sure the summary is concise, so I should highlight the key differences between the platforms and the main findings. The user might be looking for a quick overview to decide which platform suits their needs. They might be in the construction or renewable energy industry, needing project management tools. The summary should ment

In [ ]:
from langchain.chains.summarize import load_summarize_chain


chunk_docs = [Document(page_content=chunk) for chunk in chunks]

chain = load_summarize_chain(LLM_Model, chain_type="stuff", verbose=True)
stuff_summary = chain.run(chunk_docs)

print(stuff_summary)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"MARKET RESEARCH REPORT: ANALYSIS OF DOCUMENT TRANSLATION TOOLS Evaluating Leading Solutions for Multilingual Document Translation Mah inour Mohammad
Introduction This market research report analyzes competitors offering document translation tools that support PDF, Word, Excel, and scanned images while preserving layout and formatting. The focus is on tools that handle Arabic, French, and English languages, catering to both B2B and B2C markets. The key features evaluated include layout preservation, Arabic support and quality, translation accuracy and speed, pricing model, and Optical Character Recognition (OCR) support. To assess these tools, a series of test cases were conducted for each language, including: 1. Text -based documents: Evaluating basic translation accuracy, layout preservation, handling of number lists, bullet points, and